In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.layers import Dense, Dropout, Input, BatchNormalization
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("drive/MyDrive/Colab Notebooks/CERN Project 2022/Generated.csv", delimiter=' ', header=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.sum(axis=1)

0       6.507204
1       7.995110
2       1.494364
3       7.881509
4       6.515179
          ...   
4995    7.499999
4996    8.258948
4997    8.674765
4998    4.155492
4999    6.995077
Length: 5000, dtype: float64

In [ ]:
def build_generator():
    #Initializing a neural network
    generator = Sequential()
    
    #Adding an Input layer to the network
    generator.add(Dense(units=256, input_dim=121))    

    #Activating the layer with LeakyReLU activation function
    generator.add(LeakyReLU(0.2))

    #Applying Batch Normalization 
    generator.add(BatchNormalization(momentum=0.8))

    #Adding a second layer	
    generator.add(Dense(units=512))
    
    #Adding a third layer	
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    #The output layer with 121(11x11) nodes
    generator.add(Dense(units=121, activation='tanh'))
    
    #Compiling the Generator Network with loss and optimizer functions
    generator.compile(loss='binary_crossentropy', optimizer= tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
    
    return generator

In [ ]:
def build_discriminator():
    #Initializing a neural network
    discriminator=Sequential()
    
    #Adding an Input layer to the network
    discriminator.add(Dense(units=1024, input_dim= 121))
    
    #Activating the layer with LeakyReLU activation function
    discriminator.add(LeakyReLU(0.2))
    
    #Adding a dropout layer to reduce overfitting
    discriminator.add(Dropout(0.2))
       
    #Adding a second layer
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    #Adding a third layer	 
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    #Adding a forth layer
    discriminator.add(Dense(units=128))
    discriminator.add(LeakyReLU(0.2))
    
    #Adding the output layer with sigmoid activation
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    #Compiling the Discriminator Network with loss and optimizer functions
    discriminator.compile(loss='binary_crossentropy', optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
    
    return discriminator

In [ ]:
def gan_net(generator, discriminator):
  
    #Setting the trainable parameter of discriminator to False
    discriminator.trainable=False
    
    #Instantiates a Keras tensor of shape 121 (Noise shape)
    inp = Input(shape=(121,))
    
    #Feeds the input noise to the generator and stores the output in X
    X = generator(inp)
    
    #Feeds the output from generator(X) to the discriminator and stores the result in out
    out= discriminator(X)
    
    #Creates a model include all layers required in the computation of out given inp.
    gan= Model(inputs=inp, outputs=out)
    
    #Compiling the GAN Network
    gan.compile(loss='binary_crossentropy', optimizer = 'adam')
    
    return gan

In [ ]:
def get_output(epoch, generator, dim = (11,11), figsize=(15,15)):
    #Generate a normally distributed noise of shape(11X11)
    noise= np.random.normal(loc=0, scale=1, size=[121, 121]) 
    #Generate an image for the input noise
    generated_events = generator.predict(noise)
    #Reshape the generated image 
    print("Generated 1############", generated_events)
    pd.DataFrame(generated_events).to_csv('Outputs.csv')
    generated_events = generated_events.reshape(121,11,11)
    print("Generated 2############", generated_events)
    

In [ ]:

def train(X_train, epochs=5, batch_size=11):
    
    
    #Initializing the GAN 
    generator= build_generator()
    discriminator= build_discriminator()
    gan = gan_net(generator,discriminator)
    
    
    # Training the model for specified epochs
    
    for epoch in range(1,epochs+1 ):
        print("###### @ Epoch ", epoch)
        
        #tqdm module helps to generate a status bar for training 
        for _ in tqdm(range(batch_size)):
          
            #Random noise with size batch_sizex121
            noise= np.random.normal(0,1, [batch_size, 121])
            print(noise.shape)
            #Generating images from noise
            generated_events = generator.predict(noise)
            
            #taking random images from the training set 
            image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            print(image_batch.shape)
            #Creating a new training set with real and fake images 
            X= np.concatenate([image_batch, generated_events])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)  
            y_dis[:batch_size]=1.0 # label for real images
            
            #Training the discriminator with real and generated images
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Labelling the generated images as real images(1) to trick the discriminator
            noise= np.random.normal(0,1, [batch_size, 121])
            y_gen = np.ones(batch_size)
            
            #Freezing the weights of the discriminator while training generator
            discriminator.trainable=False
            
            #Training the gan network
            gan.train_on_batch(noise, y_gen)
        
        #Plotting the images for every 10 epochs
        
        get_output(epoch,generator,dim = (11,11) , figsize=(15,15))
X_train = np.array(df)
train(X_train,epochs = 5, batch_size = 121)

In [ ]:
res = pd.read_csv("Outputs.csv")
res.head(10)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,111,112,113,114,115,116,117,118,119,120
0,0,0.015071,0.010952,0.018891,-0.018515,-0.071488,0.016894,-0.027641,-0.020733,0.018405,...,-0.034115,0.000342,-0.027922,0.023028,-0.002046,-0.013806,-0.015839,0.001997,-0.033981,0.056122
1,1,-0.011850,-0.011690,0.016525,0.029866,-0.002160,0.016932,0.012532,-0.031254,-0.007011,...,-0.033182,0.015674,0.002660,-0.033469,0.020628,-0.012383,-0.019630,-0.024617,-0.031868,-0.021502
2,2,0.015190,-0.020021,0.007532,-0.037155,0.016588,-0.008762,0.004811,-0.009888,0.009369,...,-0.015375,0.035312,-0.003711,-0.006147,0.030369,-0.052586,0.032331,0.001111,0.009034,0.005187
3,3,0.031383,0.003936,0.022846,-0.007238,-0.008031,0.014275,0.010273,-0.002392,0.025251,...,-0.025289,0.002589,0.011716,-0.005954,-0.034890,-0.026590,0.008433,0.025281,-0.017947,0.040458
4,4,-0.008622,-0.039028,0.030585,-0.032696,-0.008282,-0.024523,0.042093,-0.040123,-0.029286,...,0.046739,0.021420,-0.024855,-0.032381,-0.009471,-0.030616,0.009017,-0.007852,-0.019448,-0.021809
5,5,0.016913,-0.023836,-0.013497,0.021442,-0.024955,0.032058,-0.034817,-0.017823,-0.034421,...,-0.015421,-0.019003,0.003739,-0.041440,-0.027725,0.019796,0.020079,-0.026320,-0.027162,0.001231
6,6,0.022047,0.023193,-0.028710,-0.126895,0.010754,0.029307,0.012171,-0.057559,-0.009771,...,0.012648,-0.002374,-0.061593,-0.032385,0.012360,-0.015748,-0.015938,-0.035812,-0.015816,0.037248
7,7,0.007240,0.004819,-0.027021,-0.060905,-0.015115,0.057026,0.000491,-0.022031,-0.047524,...,-0.034570,0.013977,-0.050372,-0.038604,-0.001788,-0.011997,0.011527,-0.065268,-0.049394,0.022017
8,8,0.043992,-0.030211,-0.053853,-0.039135,-0.011693,0.017707,0.043788,-0.006442,-0.067514,...,-0.013138,0.017775,-0.055295,-0.082315,0.003994,-0.029355,-0.023630,-0.013376,-0.035817,0.032276
9,9,0.027411,0.000644,-0.007888,-0.047016,-0.019798,0.040458,-0.000280,-0.063829,-0.016013,...,-0.062598,-0.023053,-0.009402,-0.029430,0.002823,-0.001613,0.021860,-0.048198,-0.044263,0.002348


In [ ]:
df.head(10)

,0.000000000000000000e+00,0.000000000000000000e+00.1,0.000000000000000000e+00.2,0.000000000000000000e+00.3,0.000000000000000000e+00.4,0.000000000000000000e+00.5,0.000000000000000000e+00.6,0.000000000000000000e+00.7,0.000000000000000000e+00.8,0.000000000000000000e+00.9,...,0.000000000000000000e+00.82,0.000000000000000000e+00.83,0.000000000000000000e+00.84,0.000000000000000000e+00.85,0.000000000000000000e+00.86,0.000000000000000000e+00.87,0.000000000000000000e+00.88,0.000000000000000000e+00.89,0.000000000000000000e+00.90,0.000000000000000000e+00.91
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
res.sum(axis=0)

Unnamed: 0    7260.000000
0                2.380540
1               -0.266763
2               -0.692700
3               -4.254777
                 ...     
116             -1.285970
117             -2.210133
118             -2.082794
119             -3.874245
120              1.599638
Length: 122, dtype: float64

plot images from csv
convert negative values to 0